#Assignment 6
Same as Assignment 5, using *Pandas*.

##Input data in the form of **Pandas Series** and **Pandas DataFrame**

In [123]:
import pandas as pd

In [124]:
area = pd.Series([1*0.25,1*0.22,1*0.015], ["A_sec","A_brick","A_updown"])
R_name = ["R_in","R_foam","R_plasterLeft","R_plasterUp","R_brick","R_plasterDown","R_plasterRight","R_out"]
R_type = pd.Series(["conv","cond","cond","cond","cond","cond","cond","conv"], index=R_name)
R_config = pd.Series(["serie","serie","serie","parallel","parallel","parallel","serie","serie"], index=R_name)
A = pd.Series([area["A_sec"],area["A_sec"],area["A_sec"],area["A_updown"],area["A_brick"],area["A_updown"],area["A_sec"],area["A_sec"]], index=R_name)
L = pd.Series([None,0.03,0.02,0.16,0.16,0.16,0.02,None], index=R_name)
k = pd.Series([None,0.026,0.22,0.22,0.72,0.22,0.22,None], index=R_name)
h = pd.Series([10,None,None,None,None,None,None,40], index=R_name)
R = pd.Series([0,0,0,0,0,0,0,0], index=R_name)

In [125]:
R_lol = [R_type, R_config, A, L, k, h, R]
R_DF = pd.DataFrame(R_lol, index=["Type","Configuration","Area (m^2)","Length (m)","k (W/m/K)","h (W/m^2/K)","R (K/W)"], columns=R_name)
R_DF = R_DF.transpose()
R_DF

,Type,Configuration,Area (m^2),Length (m),k (W/m/K),h (W/m^2/K),R (K/W)
R_in,conv,serie,0.25,NaN,NaN,10,0
R_foam,cond,serie,0.25,0.03,0.026,NaN,0
R_plasterLeft,cond,serie,0.25,0.02,0.22,NaN,0
R_plasterUp,cond,parallel,0.015,0.16,0.22,NaN,0
R_brick,cond,parallel,0.22,0.16,0.72,NaN,0
R_plasterDown,cond,parallel,0.015,0.16,0.22,NaN,0
R_plasterRight,cond,serie,0.25,0.02,0.22,NaN,0
R_out,conv,serie,0.25,NaN,NaN,40,0


##Calculation of each thermal resistances and updating DataFrame

Convective thermal resistances:

In [126]:
isConv = R_DF.loc[:,"Type"] == "conv"
R_DF.loc[isConv,"R (K/W)"] = 1/R_DF.loc[isConv,"h (W/m^2/K)"]/R_DF.loc[isConv, "Area (m^2)"]

Conductive thermal resistances:

In [127]:
isCond = R_DF.loc[:,"Type"] == "cond"
R_DF.loc[isCond,"R (K/W)"] = R_DF.loc[isCond,"Length (m)"]/R_DF.loc[isCond,"k (W/m/K)"]/R_DF.loc[isCond,"Area (m^2)"]

Updated DataFrame with each thermal resistance value of different thermal resistances:

In [128]:
R_DF

,Type,Configuration,Area (m^2),Length (m),k (W/m/K),h (W/m^2/K),R (K/W)
R_in,conv,serie,0.25,NaN,NaN,10,0.4
R_foam,cond,serie,0.25,0.03,0.026,NaN,4.61538
R_plasterLeft,cond,serie,0.25,0.02,0.22,NaN,0.363636
R_plasterUp,cond,parallel,0.015,0.16,0.22,NaN,48.4848
R_brick,cond,parallel,0.22,0.16,0.72,NaN,1.0101
R_plasterDown,cond,parallel,0.015,0.16,0.22,NaN,48.4848
R_plasterRight,cond,serie,0.25,0.02,0.22,NaN,0.363636
R_out,conv,serie,0.25,NaN,NaN,40,0.1


##Calculation of total thermal resistance

Calculating total resistance of the *parallel* resistances:

In [129]:
isParallel = R_DF.loc[:,"Configuration"] == "parallel"
S1 = 1/R_DF.loc[isParallel,"R (K/W)"]
R_parallel_tot = (S1.sum()) ** -1

Calculating total resistance of the *series* resistances:

In [130]:
isSerie = R_DF.loc[:,"Configuration"] == "serie"
S2 = R_DF.loc[isSerie,"R (K/W)"]
R_serie_tot = S2.sum()

Calculating *total* resistance:

In [131]:
R_tot = R_parallel_tot + R_serie_tot
print("The value of total thermal resistance is: " + str(round(R_tot,4)) + " [K/W]")

The value of total thermal resistance is: 6.8124 [K/W]


##Calculation of the rate of heat transfer through the wall

In [132]:
T = pd.Series([20,-10], ["in","out"])
A_wall = 3*5
A_unit = 0.25*1
Q_dot = (A_wall/A_unit) * (T["in"] - T["out"])/R_tot
print("The rate of heat transfer through the wall is: " + str(round(Q_dot,2)) + " [W]")

The rate of heat transfer through the wall is: 264.23 [W]
